In [ ]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
from ultralytics import YOLO
import os

app = Flask(__name__)

# 모델 불러오기: 학습된 모델의 경로를 지정해야 합니다.
model = YOLO('학습된 모델의 경로')  # 여기에 학습된 모델의 경로를 입력하세요.

@app.route("/predict", methods=["POST"])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part in the request'}), 400

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No file selected for uploading'}), 400

    filename = secure_filename(file.filename)
    # 이미지 저장 경로: 이미지를 임시로 저장할 디렉토리를 지정해야 합니다.
    file_path = os.path.join('임시 이미지 저장 디렉토리', filename)  # 여기에 이미지를 임시로 저장할 디렉토리를 입력하세요.
    file.save(file_path)

    # 예측 수행
    results = model(file_path)
    
    predictions = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            predictions.append({
                'class': box.cls,
                'confidence': box.conf,
                'x': box.x,
                'y': box.y,
                'width': box.width,
                'height': box.height
            })

    return jsonify({'predictions': predictions})

if __name__ == "__main__":
    app.run(debug=True)